# Building a tokenizer, block by block

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
# Menginstal pustaka untuk memproses dataset dan tokenizer.
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from datasets import load_dataset
 # Memuat dataset wikitext-2
dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")


def get_training_corpus():
    for i in range(0, len(dataset), 1000):
      # Menghasilkan korpus pelatihan dalam batch 1000 teks.
        yield dataset[i : i + 1000]["text"]

In [ ]:
# Menyimpan dataset ke file teks wikitext-2.txt.
with open("wikitext-2.txt", "w", encoding="utf-8") as f:
    for i in range(len(dataset)):
        f.write(dataset[i]["text"] + "\n")

In [ ]:
# Mengimpor komponen untuk membangun tokenizer.
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [ ]:
# Membuat tokenizer WordPiece.
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

In [ ]:
# Menambahkan urutan normalisasi.
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [ ]:
# Menampilkan teks setelah normalisasi.
print(tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


In [ ]:
# Menggunakan pre-tokenizer BertPreTokenizer
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

In [ ]:
# Mengganti pre-tokenizer dengan pemisah spasi.
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [ ]:
# Menguji pre-tokenizer.
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[('Let', (0, 3)),
 ("'", (3, 4)),
 ('s', (4, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre', (14, 17)),
 ('-', (17, 18)),
 ('tokenizer', (18, 27)),
 ('.', (27, 28))]

In [ ]:
# Pre-tokenizer untuk memisahkan spasi.
pre_tokenizer = pre_tokenizers.WhitespaceSplit()
# Menguji pre-tokenizer WhitespaceSplit.
pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[("Let's", (0, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre-tokenizer.', (14, 28))]

In [ ]:
pre_tokenizer = pre_tokenizers.Sequence(
    # Pre-tokenizer dengan pemisah spasi dan tanda baca.
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)
# Menguji pre-tokenizer sequence.
pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

[('Let', (0, 3)),
 ("'", (3, 4)),
 ('s', (4, 5)),
 ('test', (6, 10)),
 ('my', (11, 13)),
 ('pre', (14, 17)),
 ('-', (17, 18)),
 ('tokenizer', (18, 27)),
 ('.', (27, 28))]

In [ ]:
# Daftar token khusus.
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
# Melatih tokenizer WordPiece.
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [ ]:
# Melatih tokenizer dengan korpus.
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
# Mengatur model WordPiece.
tokenizer.model = models.WordPiece(unk_token="[UNK]")
# Melatih tokenizer menggunakan file teks.
tokenizer.train(["wikitext-2.txt"], trainer=trainer)

In [ ]:
# Mengencode teks menggunakan tokenizer.
encoding = tokenizer.encode("Let's test this tokenizer.")
 # Menampilkan token yang dihasilkan.
print(encoding.tokens)

['let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.']


In [ ]:
 # Mendapatkan ID token [CLS].
cls_token_id = tokenizer.token_to_id("[CLS]")
# Mendapatkan ID token [SEP].
sep_token_id = tokenizer.token_to_id("[SEP]")
 # Menampilkan ID token khusus.
print(cls_token_id, sep_token_id)

2 3


In [ ]:
# Menambahkan post-processor untuk template pemrosesan.
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [ ]:
# Mengencode pasangan teks.
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.', '[SEP]']


In [ ]:
# Mengencode pasangan teks.
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences.")
print(encoding.tokens)
print(encoding.type_ids)

['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '...', '[SEP]', 'on', 'a', 'pair', 'of', 'sentences', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# Menambahkan decoder WordPiece.
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [ ]:
# Mendecode token kembali ke teks.
tokenizer.decode(encoding.ids)

"let ' s test this tokenizer... on a pair of sentences."

In [ ]:
# Menyimpan tokenizer ke file JSON.
tokenizer.save("tokenizer.json")

In [ ]:
# Memuat tokenizer dari file JSON.
new_tokenizer = Tokenizer.from_file("tokenizer.json")

In [ ]:
from transformers import PreTrainedTokenizerFast
# Membungkus tokenizer untuk digunakan dengan Transformers.
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from transformers import BertTokenizerFast

wrapped_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

In [ ]:
# Membuat tokenizer dengan model BPE.
tokenizer = Tokenizer(models.BPE())

In [ ]:
# Menambahkan pre-tokenizer ByteLevel.
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

In [ ]:
# Menguji pre-tokenizer ByteLevel
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test pre-tokenization!")

[('Let', (0, 3)),
 ("'s", (3, 5)),
 ('Ġtest', (5, 10)),
 ('Ġpre', (10, 14)),
 ('-', (14, 15)),
 ('tokenization', (15, 27)),
 ('!', (27, 28))]

In [ ]:
# Melatih tokenizer BPE.
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
 # Melatih tokenizer dengan iterator korpus.
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
# Mengatur model BPE.
tokenizer.model = models.BPE()
# Melatih tokenizer dengan file teks.
tokenizer.train(["wikitext-2.txt"], trainer=trainer)

In [ ]:
# Mengencode teks dengan tokenizer BPE.
encoding = tokenizer.encode("Let's test this tokenizer.")
 # Menampilkan token BPE yang dihasilkan.
print(encoding.tokens)

['L', 'et', "'", 's', 'Ġtest', 'Ġthis', 'Ġto', 'ken', 'izer', '.']


In [ ]:
 # Menambahkan post-processor ByteLevel.
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)

In [ ]:
sentence = "Let's test this tokenizer."
# Mengencode kalimat dengan post-processor.
encoding = tokenizer.encode(sentence)
# Mendapatkan rentang karakter token.
start, end = encoding.offsets[4]
 # Menampilkan token asli berdasarkan rentang karakter.
sentence[start:end]

' test'

In [ ]:
# Menambahkan decoder ByteLevel.
tokenizer.decoder = decoders.ByteLevel()

In [ ]:
# Mendecode token kembali ke teks asli.
tokenizer.decode(encoding.ids)

"Let's test this tokenizer."

In [ ]:
from transformers import PreTrainedTokenizerFast
# Membungkus tokenizer BPE untuk Transformers.
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>",
)

In [ ]:
from transformers import GPT2TokenizerFast
# Membungkus tokenizer sebagai GPT2TokenizerFast.
wrapped_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

In [ ]:
# Membuat tokenizer dengan model Unigram.
tokenizer = Tokenizer(models.Unigram())

In [ ]:
from tokenizers import Regex
# Menormalkan teks dengan model

tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace("``", '"'),
        normalizers.Replace("''", '"'),
        normalizers.NFKD(),
        normalizers.StripAccents(),
        normalizers.Replace(Regex(" {2,}"), " "),
    ]
)

In [ ]:
# Menggunakan pre-tokenizer Metaspace.
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

In [ ]:
# Menguji pre-tokenizer Metaspace.
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test the pre-tokenizer!")

[("▁Let's", (0, 5)),
 ('▁test', (5, 10)),
 ('▁the', (10, 14)),
 ('▁pre-tokenizer!', (14, 29))]

In [ ]:
# Melatih tokenizer Unigram dengan korpus.
special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]
trainer = trainers.UnigramTrainer(
    vocab_size=25000, special_tokens=special_tokens, unk_token="<unk>"
)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [ ]:
# Mengatur model Unigram.
tokenizer.model = models.Unigram()
tokenizer.train(["wikitext-2.txt"], trainer=trainer)

In [ ]:
# Mengencode teks dengan tokenizer Unigram.
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['▁Let', "'", 's', '▁test', '▁this', '▁to', 'ken', 'izer', '.']


In [ ]:
# Mendapatkan ID token <cls>.
cls_token_id = tokenizer.token_to_id("<cls>")
sep_token_id = tokenizer.token_to_id("<sep>")
print(cls_token_id, sep_token_id)

0 1


In [ ]:
tokenizer.post_processor = processors.TemplateProcessing(
    single="$A:0 <sep>:0 <cls>:2",
    pair="$A:0 <sep>:0 $B:1 <sep>:1 <cls>:2",
    special_tokens=[("<sep>", sep_token_id), ("<cls>", cls_token_id)],
)

In [ ]:
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences!")
print(encoding.tokens)
print(encoding.type_ids)

['▁Let', "'", 's', '▁test', '▁this', '▁to', 'ken', 'izer', '.', '.', '.', '<sep>', '▁', 'on', '▁', 'a', '▁pair', '▁of', '▁sentence', 's', '!', '<sep>', '<cls>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]


In [ ]:
tokenizer.decoder = decoders.Metaspace()

In [ ]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="left",
)

In [ ]:
from transformers import XLNetTokenizerFast

wrapped_tokenizer = XLNetTokenizerFast(tokenizer_object=tokenizer)